```@meta
CurrentModule = CounterfactualExplanations 
```

# MNIST

In this example we will see how different counterfactual generators can be used to explain deep learning models for image classification. In particular, we will look at MNIST data and visually inspect how the different generators perturb images of handwritten digits in order to change the predicted label to a target label. @fig-samples shows a random sample of handwritten digits.


In [ ]:
#| echo: false
using Pkg; Pkg.activate("docs")
using CounterfactualExplanations, Plots, MLDatasets
using MLDatasets: convert2image
using BSON: @save, @load
theme(:wong)
using Logging
disable_logging(Logging.Info)
include("dev/utils.jl") # some plotting functions
www_path = "docs/src/examples/image/www"

In [ ]:
#| echo: false
data_train = MNIST(:train)
train_x, train_y = data_train[:]
input_dim = prod(size(train_x[:,:,1]))
using Images, Random, StatsBase
Random.seed!(1)
n_samples = 10
samples = train_x[:,:,sample(1:end, n_samples, replace=false)]
mosaic = mosaicview([convert2image(MNIST, samples[:,:,i]) for i ∈ 1:n_samples]...,ncol=Int(n_samples/2))
plt = plot(mosaic, size=(500,260), axis=nothing, background=:transparent)
savefig(plt, joinpath(www_path, "mnist_samples.png"))

![A few random handwritten digits.](www/mnist_samples.png){#fig-samples}

## Pre-trained classifiers

Next we will load two pre-trained deep-learning classifiers:

1. Simple MLP - `model`
2. Deep ensemble - `ensemble`


In [ ]:
using Flux
using CounterfactualExplanations.Data: mnist_data, mnist_model, mnist_ensemble
X, ys = mnist_data()
model = mnist_model()
ensemble = mnist_ensemble()

The following code just prepares the models to be used with `CounterfactualExplanations.jl`:


In [ ]:
using CounterfactualExplanations, CounterfactualExplanations.Models
import CounterfactualExplanations.Models: logits, probs # import functions in order to extend

M = FluxModel(model, likelihood=:classification_multi)
M_ensemble=FluxEnsemble(ensemble, likelihood=:classification_multi)

## Generating counterfactuals

We will look at four different approaches here: 

1. Generic approach for the MLP [@wachter2017counterfactual].
2. Greedy approach for the MLP.
3. Generic approach for the deep ensemble.
4. Greedy approach for the deep ensemble [@schut2021generating].

They can be implemented using the `GenericGenerator` and the `GreedyGenerator`.

### Turning a 9 into a 4

We will start with an example that should yield intuitive results: the process of turning a handwritten 9 in @fig-nine into a 4 is straight-forward for a human - just erase the top part. Let's see how the different algorithmic approaches perform. First, we preprocess the data below, where we impose that the features (pixel values) are constrained to the follwoing domain: $\mathcal{X}=[0,1]\subset\mathbb{R}$.


In [ ]:
counterfactual_data = CounterfactualData(X,ys;domain=(0,1))

Next we choose a random sample for which we will generate counterfactuals in the following:


In [ ]:
# Randomly selected factual:
using Random
Random.seed!(42)
x = reshape(X[:,rand(findall(Flux.onecold(ys,0:9).==9))],input_dim,1)
target = 5
γ = 0.95

In [ ]:
#| echo: false
using Images
input_dim = size(X)[1]
img = convert2image(MNIST, reshape(x,Int(√(input_dim)),Int(√(input_dim))))
plt_orig = plot(img, title="Original", axis=nothing)
savefig(plt_orig, joinpath(www_path, "mnist_original.png"))

![A random handwritten 9.](www/mnist_original.png){#fig-nine}

The code below implements the four different approaches one by one. @fig-example shows the resulting counterfactuals. In every case the desired label switch is achieved, that is the corresponding classifier classifies the counterfactual as a four. But arguably from a human perspective only the counterfactuals for the deep ensemble look like a 4. For the MLP, both the generic and the greedy approach generate counterfactuals that look much like adversarial examples.


In [ ]:
# Generic - MLP
generator = GenericGenerator(;loss=:logitcrossentropy)
counterfactual = generate_counterfactual(x, target, counterfactual_data, M, generator; γ=γ)
img = convert2image(MNIST,reshape(counterfactual(counterfactual),Int(√(input_dim)),Int(√(input_dim))))
plt_wachter = plot(img, title="MLP - Wachter")

# Generic - Deep Ensemble
counterfactual = generate_counterfactual(x, target, counterfactual_data, M_ensemble, generator; γ=γ)
img = convert2image(MNIST,reshape(counterfactual(counterfactual),Int(√(input_dim)),Int(√(input_dim))))
plt_wachter_de = plot(img, title="Ensemble - Wachter")

# Greedy - MLP
generator = GreedyGenerator(;loss=:logitcrossentropy)
counterfactual = generate_counterfactual(x, target, counterfactual_data, M, generator; γ=γ)
img = convert2image(MNIST,reshape(counterfactual(counterfactual),Int(√(input_dim)),Int(√(input_dim))))
plt_greedy = plot(img, title="MLP - Greedy")

# Greedy - Deep Ensemble
counterfactual = generate_counterfactual(x, target, counterfactual_data, M_ensemble, generator; γ=γ)
img = convert2image(MNIST,reshape(counterfactual(counterfactual),Int(√(input_dim)),Int(√(input_dim))))
plt_greedy_de = plot(img, title="Ensemble - Greedy")

plt_list = [plt_orig, plt_wachter, plt_greedy, plt_wachter_de, plt_greedy_de]
plt = plot(plt_list...,layout=(1,length(plt_list)),axis=nothing, size=(1200,240))
savefig(plt, joinpath(www_path, "MNIST_9to4.png"))

![Counterfactual explanations for MNIST data: turning a 9 into a 4](www/MNIST_9to4.png){#fig-example}


In [ ]:
#| echo: false
using Random

# Single:
function from_digit_to_digit(from, to, generator, model; γ=0.95, x=X, y=ys, seed=1234, T=1000)

    Random.seed!(seed)

    candidates = findall(onecold(y,0:9).==from)
    x = Flux.unsqueeze(x[rand(candidates)],2)
    target = to + 1
    counterfactual = generate_counterfactual(x, target, counterfactual_data, v_mod, v_gen; T=T)

    return recourse
end

# Multiple:
function from_digit_to_digit(from, to, generator::Dict, model::Dict; γ=0.95, x=X, y=ys, seed=1234, T=1000)

    Random.seed!(seed)

    candidates = findall(onecold(y,0:9).==from)
    x = Flux.unsqueeze(x[:,rand(candidates)],2)
    target = to + 1
    counterfactuals = Dict()

    for (k_gen,v_gen) ∈ generators
        for (k_mod,v_mod) ∈ models 
            k = k_mod * " - " * k_gen
            counterfactuals[k] = generate_counterfactual(x, target, counterfactual_data, v_mod, v_gen; T=T)
        end
    end

    return counterfactuals
end

In [ ]:
#| echo: false
generators = Dict(
    "Wachter" => GenericGenerator(;loss=:logitcrossentropy),
    "Greedy" => GreedyGenerator(;loss=:logitcrossentropy)
)
models = Dict("MLP" => M, "Ensemble" => M_ensemble)

In [ ]:
#| echo: false
from = 3
to = 8
counterfactuals = from_digit_to_digit(from,to,generators,models)
plts =  first(values(counterfactuals)).x |> x -> plot(convert2image(reshape(x,Int(√(input_dim)),Int(√(input_dim)))),title="Original")
plts = vcat(plts, [plot(convert2image(reshape(v.x′,Int(√(input_dim)),Int(√(input_dim)))),title=k) for (k,v) in counterfactuals])
plt = plot(plts...,layout=(1,length(plts)),axis=nothing, size=(1200,240))
savefig(plt, joinpath(www_path, "MNIST_$(from)to$(to).png"))

In [ ]:
#| echo: false
from = 7
to = 2
counterfactuals = from_digit_to_digit(from,to,generators,models)
plts =  first(values(counterfactuals)).x |> x -> plot(convert2image(reshape(x,Int(√(input_dim)),Int(√(input_dim)))),title="Original")
plts = vcat(plts, [plot(convert2image(reshape(v.x′,Int(√(input_dim)),Int(√(input_dim)))),title=k) for (k,v) in counterfactuals])
plt = plot(plts...,layout=(1,length(plts)),axis=nothing, size=(1200,240))
savefig(plt, joinpath(www_path, "MNIST_$(from)to$(to).png"))

In [ ]:
#| echo: false
from = 1
to = 7
counterfactuals = from_digit_to_digit(from,to,generators,models)
plts =  first(values(counterfactuals)).x |> x -> plot(convert2image(reshape(x,Int(√(input_dim)),Int(√(input_dim)))),title="Original")
plts = vcat(plts, [plot(convert2image(reshape(v.x′,Int(√(input_dim)),Int(√(input_dim)))),title=k) for (k,v) in counterfactuals])
plt = plot(plts...,layout=(1,length(plts)),axis=nothing, size=(1200,240))
savefig(plt, joinpath(www_path, "MNIST_$(from)to$(to).png"))

### References
